In [ ]:
import pandas as pd
df = pd.read_csv('queryset_CNN.csv')
print(df.shape)
print(df.dtypes)

(32, 3)
doc_id         int64
author_id      int64
authorList    object
dtype: object


In [ ]:

for index, row in df.iterrows():
    doc_id = row.doc_id

    author_id = row.author_id

    import ast
    authorList = ast.literal_eval(row.authorList)
    
    candidate = len(authorList)
    
    algo = "svc"
    
    test = algo # change before run
    
    level = "word"
    
    iterations = 10

    dropout = 0.5

    samples = 3200

    dimensions = 200

    loc = authorList.index(author_id)

    printstate = (("doc_id = %s, candidate = %s, ") % (str(doc_id), str(candidate)))
    printstate += (("dimensions = %s, samples = %s, ") % (str(dimensions), str(samples)))
    printstate += (("\niterations = %s, dropout = %s, test = %s") % (str(iterations), str(dropout), str(test)))

    print("Current test: %s" % (str(printstate)))
    from sshtunnel import SSHTunnelForwarder
    with SSHTunnelForwarder(('144.214.121.15', 22),
                            ssh_username='ninadt',
                            ssh_password='Ninad123',
                            remote_bind_address=('localhost', 3306),
                            local_bind_address=('localhost', 3300)):
        import UpdateDB as db
        case = db.checkOldCNNDiff(doc_id = doc_id, candidate = candidate, dimensions = dimensions,
                                  samples = samples, iterations = iterations, dropout = dropout,
                                  test = test, port = 3300)

    if case == False:

        print("Running: %12s" % (str(printstate)))

        import StyloNeuralML as Stylo
        (labels_index, train_acc, val_acc, samples) = Stylo.getResults(
            doc_id = doc_id, authorList = authorList[:], algo = algo,
            level = level, glove = '../../glove/', dimensions = dimensions, 
            samples = samples, nb_epoch = iterations, dropout = dropout, batch_size = 10 )
        
        (predY, testY) = Stylo.getTestResults(
            doc_id = doc_id, authorList = authorList[:], labels_index = labels_index, algo = algo,
            level = level, glove = '../../glove/', dimensions = dimensions, 
            samples = samples, nb_epoch = iterations, dropout = dropout, batch_size = 10 )
            
        loc = testY
        
        test_acc = 0.0
        
        test_acc = predY[loc]

        test_bin = 0

        if(predY.tolist().index(max(predY)) == testY):
            test_bin = 1
        
        from sshtunnel import SSHTunnelForwarder
        with SSHTunnelForwarder(('144.214.121.15', 22),
                                ssh_username='ninadt',
                                ssh_password='Ninad123',
                                remote_bind_address=('localhost', 3306),
                                local_bind_address=('localhost', 3300)):
            import UpdateDB as db
            case = db.updateresultOldCNNDiff(doc_id = doc_id, candidate = candidate, dimensions = dimensions,
                                             samples = samples, iterations = iterations, dropout = dropout, 
                                             train_acc = train_acc, val_acc = val_acc,
                                             test_acc = test_acc, test_bin = test_bin,
                                             test = test, port = 3300)
        del Stylo

        from keras import backend as K
        K.clear_session()
        
        import time
        time.sleep(10)

        from IPython.display import clear_output

        clear_output()

    else:
        print("Skipped: %12s" % (str(printstate)))

# import pandas as pd
# df = pd.DataFrame(output)
# df.to_csv("styloout.csv", index = False, encoding='utf-8')

import time
time.sleep(10)

Current test: doc_id = 85, candidate = 3, dimensions = 200, samples = 3200, 
iterations = 10, dropout = 0.5, test = svc
Execution completed
Running: doc_id = 85, candidate = 3, dimensions = 200, samples = 3200, 
iterations = 10, dropout = 0.5, test = svc


Using TensorFlow backend.


Level = Word
File used: glove.6B.200d.txt
Found 400000 word vectors.
Execution completed
Read completed
Number of rows: 124
author_id       int64
doc_content    object
dtype: object
Data Frame created: Shape: (11578, 2)
Author:    44  Size:  4746
Author:    64  Size:  5106
Author:    82  Size:  1726
Min: 1726
Max: 5106
Authors [44, 64, 82].
Found 5178 texts.
Found 5178 labels.
Found 46523 unique tokens.
Shape of data tensor: (5178, 1000)
Shape of label tensor: (5178, 3)
Done compiling.
Train on 4142 samples, validate on 1036 samples
Epoch 1/10
4142/4142 [==============================] - 53s - loss: 0.8287 - acc: 0.6021 - val_loss: 0.5147 - val_acc: 0.7828
Epoch 2/10
4142/4142 [==============================] - 47s - loss: 0.3804 - acc: 0.8532 - val_loss: 0.3589 - val_acc: 0.8668
Epoch 3/10
4142/4142 [==============================] - 47s - loss: 0.2355 - acc: 0.9143 - val_loss: 0.2388 - val_acc: 0.9180
Epoch 4/10
4142/4142 [==============================] - 44s - loss: 0.1412 - acc: 0

In [ ]:
%tb